# Time Dependent Evolution

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from quantum_tools import *
from experimental import *
from spam import *
from timedependent_channels import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=5, suppress=True)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Spin-Spin Hamiltonian with Local Fourier Expansion Pulses

### Fit CNOT

In [6]:
T = 1
U = tf.convert_to_tensor([[1, 0, 0, 0],
                            [0, 1, 0, 0],
                            [0, 0, 0, 1],
                            [0, 0, 1, 0]], dtype=tf.complex128)

channel_target = ChoiMapStatic(U, mode="unitary")
choi_target = channel_target.choi
n = 2
d = 2**n

tf.random.set_seed(42)
np.random.seed(42)

H_model = SpinSpin(degree=2)
jump_operator = JumpOperator(4, trainable=False)

lindblad_model = LindbladGenerator(hamiltonian = H_model, 
                                   jump_operator = jump_operator,
                                   gamma = 0.0)

channel_model = MagnusPropagator(liouvillian=lindblad_model, grid_size=200)
print(tf.math.reduce_sum(tf.abs(channel_model.choi(T) - choi_target)**2))

tf.Tensor(28.405977244987298, shape=(), dtype=float64)


In [7]:
T = 1

#optimizer = tf.optimizers.Adam(learning_rate=0.01)
for i in tqdm(range(500)):
    with tf.GradientTape() as tape:
        tape.watch(channel_model.parameter_list)
        choi_model = channel_model.choi(T)
        loss = tf.math.reduce_sum(tf.abs(channel_model.choi(T) - choi_target) ** 2)

    print(loss)
    gradients = tape.gradient(loss, channel_model.parameter_list)
    optimizer.apply_gradients(zip(gradients, channel_model.parameter_list))

  0%|          | 0/500 [00:00<?, ?it/s]

tf.Tensor(28.405977244987298, shape=(), dtype=float64)
tf.Tensor(28.085484604391247, shape=(), dtype=float64)
tf.Tensor(27.658059387795355, shape=(), dtype=float64)
tf.Tensor(27.15075072003825, shape=(), dtype=float64)
tf.Tensor(26.5783211333406, shape=(), dtype=float64)
tf.Tensor(25.956980396673202, shape=(), dtype=float64)
tf.Tensor(25.2981733188525, shape=(), dtype=float64)
tf.Tensor(24.599488147595288, shape=(), dtype=float64)
tf.Tensor(23.84557721126644, shape=(), dtype=float64)
tf.Tensor(23.015103254340968, shape=(), dtype=float64)
tf.Tensor(22.092649387917017, shape=(), dtype=float64)
tf.Tensor(21.08314512136372, shape=(), dtype=float64)
tf.Tensor(20.016172831749778, shape=(), dtype=float64)
tf.Tensor(18.940350743798533, shape=(), dtype=float64)
tf.Tensor(17.914154088307185, shape=(), dtype=float64)
tf.Tensor(16.994270948780034, shape=(), dtype=float64)
tf.Tensor(16.220788858653364, shape=(), dtype=float64)
tf.Tensor(15.604723574042477, shape=(), dtype=float64)
tf.Tensor(15.1300

In [7]:
channel_noisy = deepcopy(channel_model)
channel_noisy.liouvillian.gamma = 0.5
print(state_fidelity(choi_target, channel_model.choi(T))/16)
print(state_fidelity(choi_target, channel_noisy.choi(T))/16)

tf.Tensor(0.9999997615504477, shape=(), dtype=float64)
tf.Tensor(0.8839231254473169, shape=(), dtype=float64)


In [ ]:
T = 1

optimizer = tf.optimizers.Adam(learning_rate=0.01)
for i in tqdm(range(200)):
    with tf.GradientTape() as tape:
        tape.watch(channel_noisy.parameter_list)
        choi_model = channel_noisy.choi(T)
        loss = tf.math.reduce_sum(tf.abs(channel_noisy.choi(T) - choi_target) ** 2)

    print(loss)
    gradients = tape.gradient(loss, channel_noisy.parameter_list)
    optimizer.apply_gradients(zip(gradients, channel_noisy.parameter_list))

  0%|          | 0/200 [00:00<?, ?it/s]

tf.Tensor(0.2991020787396503, shape=(), dtype=float64)
tf.Tensor(0.3268497163585269, shape=(), dtype=float64)
tf.Tensor(0.30347930979311244, shape=(), dtype=float64)
tf.Tensor(0.30629712748099464, shape=(), dtype=float64)
tf.Tensor(0.31194088990553404, shape=(), dtype=float64)
tf.Tensor(0.30915296497310146, shape=(), dtype=float64)
tf.Tensor(0.3045588532948391, shape=(), dtype=float64)
tf.Tensor(0.30226879498065123, shape=(), dtype=float64)
tf.Tensor(0.3028696376315205, shape=(), dtype=float64)
tf.Tensor(0.3041653166221908, shape=(), dtype=float64)
tf.Tensor(0.3041150873026731, shape=(), dtype=float64)
tf.Tensor(0.3027130527650249, shape=(), dtype=float64)
tf.Tensor(0.3012999242954335, shape=(), dtype=float64)
tf.Tensor(0.3009267762347204, shape=(), dtype=float64)
tf.Tensor(0.30134681399615676, shape=(), dtype=float64)
tf.Tensor(0.30155873033329444, shape=(), dtype=float64)
tf.Tensor(0.30107742829270906, shape=(), dtype=float64)
tf.Tensor(0.3003812099540385, shape=(), dtype=float64)
tf

In [ ]:
print(state_fidelity(choi_target, channel_model.choi(T))/16)
print(state_fidelity(choi_target, channel_noisy.choi(T))/16)

tf.Tensor(0.9999997615504477, shape=(), dtype=float64)
tf.Tensor(0.8758372386006208, shape=(), dtype=float64)


In [ ]:
T = 1
U = tf.convert_to_tensor([[1, 0, 0, 0],
                            [0, 1, 0, 0],
                            [0, 0, 0, 1],
                            [0, 0, 1, 0]], dtype=tf.complex128)

channel_target = ChoiMapStatic(U, mode="unitary")
choi_target = channel_target.choi
n = 2
d = 2**n

tf.random.set_seed(42)
np.random.seed(42)

H_model = SpinSpin(degree=3)
jump_operator = JumpOperator(4, trainable=True)

lindblad_model = LindbladGenerator(hamiltonian = H_model, 
                                   jump_operator = jump_operator,
                                   gamma = 0.0)

channel_model = MagnusPropagator(liouvillian=lindblad_model, grid_size=200)
print(tf.math.reduce_sum(tf.abs(channel_model.choi(T) - choi_target)**2))

tf.Tensor(31.917988443199974, shape=(), dtype=float64)


In [ ]:
T = 1

optimizer = tf.optimizers.Adam(learning_rate=0.01)
for i in tqdm(range(200)):
    with tf.GradientTape() as tape:
        tape.watch(channel_model.parameter_list)
        choi_model = channel_model.choi(T)
        loss = tf.math.reduce_sum(tf.abs(channel_model.choi(T) - choi_target) ** 2)

    print(loss)
    gradients = tape.gradient(loss, channel_model.parameter_list)
    optimizer.apply_gradients(zip(gradients, channel_model.parameter_list))

  0%|          | 0/200 [00:00<?, ?it/s]

tf.Tensor(31.917988443199974, shape=(), dtype=float64)
tf.Tensor(31.879549092655644, shape=(), dtype=float64)
tf.Tensor(31.834301050787424, shape=(), dtype=float64)
tf.Tensor(31.78247478321377, shape=(), dtype=float64)
tf.Tensor(31.72418087545993, shape=(), dtype=float64)
tf.Tensor(31.65934394535158, shape=(), dtype=float64)
tf.Tensor(31.58760445467837, shape=(), dtype=float64)
tf.Tensor(31.50855332681749, shape=(), dtype=float64)
tf.Tensor(31.421947808342637, shape=(), dtype=float64)
tf.Tensor(31.327669217972332, shape=(), dtype=float64)
tf.Tensor(31.22568397824668, shape=(), dtype=float64)
tf.Tensor(31.116011555149186, shape=(), dtype=float64)
tf.Tensor(30.99868621363754, shape=(), dtype=float64)
tf.Tensor(30.873701335159183, shape=(), dtype=float64)
tf.Tensor(30.740934194721188, shape=(), dtype=float64)
tf.Tensor(30.600080769867116, shape=(), dtype=float64)
tf.Tensor(30.450675851147892, shape=(), dtype=float64)
tf.Tensor(30.292225742541603, shape=(), dtype=float64)
tf.Tensor(30.1243

In [ ]:
channel_noisy = deepcopy(channel_model)
channel_noisy.liouvillian.gamma = 0.5
print(state_fidelity(choi_target, channel_model.choi(T))/16)
print(state_fidelity(choi_target, channel_noisy.choi(T))/16)

tf.Tensor(0.9999997615504477, shape=(), dtype=float64)
tf.Tensor(0.8839231254473169, shape=(), dtype=float64)


In [ ]:
T = 1

optimizer = tf.optimizers.Adam(learning_rate=0.01)
for i in tqdm(range(200)):
    with tf.GradientTape() as tape:
        tape.watch(channel_noisy.parameter_list)
        choi_model = channel_noisy.choi(T)
        loss = tf.math.reduce_sum(tf.abs(channel_noisy.choi(T) - choi_target) ** 2)

    print(loss)
    gradients = tape.gradient(loss, channel_noisy.parameter_list)
    optimizer.apply_gradients(zip(gradients, channel_noisy.parameter_list))

  0%|          | 0/200 [00:00<?, ?it/s]

tf.Tensor(0.2991020787396503, shape=(), dtype=float64)
tf.Tensor(0.32578774511799274, shape=(), dtype=float64)
tf.Tensor(0.30132446846675787, shape=(), dtype=float64)
tf.Tensor(0.3030390677869881, shape=(), dtype=float64)
tf.Tensor(0.30765733289552155, shape=(), dtype=float64)
tf.Tensor(0.3038630716871165, shape=(), dtype=float64)
tf.Tensor(0.298263488690846, shape=(), dtype=float64)
tf.Tensor(0.2949976809622219, shape=(), dtype=float64)
tf.Tensor(0.2946701611284146, shape=(), dtype=float64)
tf.Tensor(0.2950642542623326, shape=(), dtype=float64)
tf.Tensor(0.2941151539879545, shape=(), dtype=float64)
tf.Tensor(0.29183177073559996, shape=(), dtype=float64)
tf.Tensor(0.2895877336949022, shape=(), dtype=float64)
tf.Tensor(0.2884411615227921, shape=(), dtype=float64)
tf.Tensor(0.2881197056506892, shape=(), dtype=float64)
tf.Tensor(0.28759386439244616, shape=(), dtype=float64)
tf.Tensor(0.2863804653136595, shape=(), dtype=float64)
tf.Tensor(0.2849861647295985, shape=(), dtype=float64)
tf.Ten

In [ ]:
print(state_fidelity(choi_target, channel_model.choi(T))/16)
print(state_fidelity(choi_target, channel_noisy.choi(T))/16)

tf.Tensor(0.9999997615504477, shape=(), dtype=float64)
tf.Tensor(0.8832164490094301, shape=(), dtype=float64)


In [ ]:
J = channel_noisy.liouvillian.JumpOperator([1])
print(tf.linalg.trace(J))

tf.Tensor([0.+0.j], shape=(1,), dtype=complex128)


In [ ]:
print(tf.linalg.trace(tf.matmul(J, J, adjoint_a=True)))

tf.Tensor([1.+0.j], shape=(1,), dtype=complex128)


In [ ]:
[print(param.numpy()) for param in channel_noisy.parameter_list]
[print(param.numpy()) for param in channel_model.parameter_list]

[ 0.32009 -1.90256  0.92831]
[-0.57666 -1.70997  1.23363 -1.03345  2.12939  0.05738 -0.11441  3.25468
  0.13797 -2.51897 -0.58319  1.52119  0.93076  0.92909  0.8544  -1.47498
 -1.09751 -0.42928]
[ 1.15258  0.24593  0.03632  0.05    -1.34788  0.96654 -0.78022  0.92374
 -1.22402 -2.14526 -2.12732 -0.73996]
[[ 0.82018 -0.45029 -0.82562  2.21404]
 [ 1.17351 -0.98223  0.43617  0.10042]
 [-0.15581 -1.37839 -0.52627  0.42364]
 [ 0.24573  0.68603 -0.31491  0.69615]]
[[-0.0829   0.3305  -0.02357  0.93595]
 [ 1.99452  0.78143  1.12252 -0.87363]
 [ 1.37224 -0.55898  0.18975  1.30706]
 [-0.0058   0.61389 -2.27839 -0.36012]]
[ 0.34881 -1.86274  0.86795]
[-0.64485 -1.71013  1.14861 -1.04768  2.03444  0.17889 -0.12125  3.20652
  0.29209 -2.50918 -0.58797  1.44163  0.94844  0.94947  0.80886 -1.46279
 -1.08694 -0.49586]
[ 1.15438  0.25876 -0.04822  0.03945 -1.3302   0.9655  -0.79513  0.97593
 -1.2466  -2.10461 -2.10313 -0.77807]
[[-0.13428 -0.38042  0.28529  1.43513]
 [ 1.13107 -0.46209 -0.0577  -0.196

[None, None, None, None, None]

## Path independence

In [ ]:
T = 1
U = tf.convert_to_tensor([[1, 0, 0, 0],
                            [0, 1, 0, 0],
                            [0, 0, 0, 1],
                            [0, 0, 1, 0]], dtype=tf.complex128)

channel_target = ChoiMapStatic(U, mode="unitary")
choi_target = channel_target.choi
n = 2
d = 2**n

tf.random.set_seed(42)
np.random.seed(42)

H_model1 = SpinSpin(degree=3)
H_model2 = SpinSpin(degree=3)
H_model3 = SpinSpin(degree=3)

lindblad_model1 = LindbladGenerator(hamiltonian = H_model1, 
                                   jump_operator = None,
                                   gamma = 0.0)
lindblad_model2 = LindbladGenerator(hamiltonian = H_model2, 
                                   jump_operator = None,
                                   gamma = 0.0)
lindblad_model3 = LindbladGenerator(hamiltonian = H_model3, 
                                   jump_operator = None,
                                   gamma = 0.0)

channel_model1 = MagnusPropagator(liouvillian=lindblad_model1, grid_size=200)
channel_model2 = MagnusPropagator(liouvillian=lindblad_model2, grid_size=200)
channel_model3 = MagnusPropagator(liouvillian=lindblad_model3, grid_size=200)
print(tf.math.reduce_sum(tf.abs(channel_model1.choi(T) - choi_target)**2))
print(tf.math.reduce_sum(tf.abs(channel_model2.choi(T) - choi_target)**2))
print(tf.math.reduce_sum(tf.abs(channel_model3.choi(T) - choi_target)**2))

tf.Tensor(31.917988443199974, shape=(), dtype=float64)
tf.Tensor(31.689685464753325, shape=(), dtype=float64)
tf.Tensor(31.26214720074752, shape=(), dtype=float64)


In [ ]:
T = 1

optimizer = tf.optimizers.Adam(learning_rate=0.01)
for i in tqdm(range(200)):
    with tf.GradientTape() as tape:
        tape.watch(channel_model1.parameter_list)
        choi_model = channel_model1.choi(T)
        loss = tf.math.reduce_sum(tf.abs(channel_model1.choi(T) - choi_target) ** 2)

    print(loss)
    gradients = tape.gradient(loss, channel_model1.parameter_list)
    optimizer.apply_gradients(zip(gradients, channel_model1.parameter_list))

optimizer = tf.optimizers.Adam(learning_rate=0.01)
for i in tqdm(range(200)):
    with tf.GradientTape() as tape:
        tape.watch(channel_model2.parameter_list)
        choi_model = channel_model2.choi(T)
        loss = tf.math.reduce_sum(tf.abs(channel_model2.choi(T) - choi_target) ** 2)

    print(loss)
    gradients = tape.gradient(loss, channel_model2.parameter_list)
    optimizer.apply_gradients(zip(gradients, channel_model2.parameter_list))


optimizer = tf.optimizers.Adam(learning_rate=0.01)
for i in tqdm(range(200)):
    with tf.GradientTape() as tape:
        tape.watch(channel_model3.parameter_list)
        choi_model = channel_model3.choi(T)
        loss = tf.math.reduce_sum(tf.abs(channel_model3.choi(T) - choi_target) ** 2)

    print(loss)
    gradients = tape.gradient(loss, channel_model3.parameter_list)
    optimizer.apply_gradients(zip(gradients, channel_model3.parameter_list))

  0%|          | 0/200 [00:00<?, ?it/s]

tf.Tensor(31.917988443199974, shape=(), dtype=float64)
tf.Tensor(31.879549092655644, shape=(), dtype=float64)
tf.Tensor(31.834301050787424, shape=(), dtype=float64)
tf.Tensor(31.78247478321377, shape=(), dtype=float64)
tf.Tensor(31.72418087545993, shape=(), dtype=float64)
tf.Tensor(31.65934394535158, shape=(), dtype=float64)
tf.Tensor(31.58760445467837, shape=(), dtype=float64)
tf.Tensor(31.50855332681749, shape=(), dtype=float64)
tf.Tensor(31.421947808342637, shape=(), dtype=float64)
tf.Tensor(31.327669217972332, shape=(), dtype=float64)
tf.Tensor(31.22568397824668, shape=(), dtype=float64)
tf.Tensor(31.116011555149186, shape=(), dtype=float64)
tf.Tensor(30.99868621363754, shape=(), dtype=float64)
tf.Tensor(30.873701335159183, shape=(), dtype=float64)
tf.Tensor(30.740934194721188, shape=(), dtype=float64)
tf.Tensor(30.600080769867116, shape=(), dtype=float64)
tf.Tensor(30.450675851147892, shape=(), dtype=float64)
tf.Tensor(30.292225742541603, shape=(), dtype=float64)
tf.Tensor(30.1243

  0%|          | 0/200 [00:00<?, ?it/s]

tf.Tensor(31.689685464753325, shape=(), dtype=float64)
tf.Tensor(31.614872787231743, shape=(), dtype=float64)
tf.Tensor(31.5281964206712, shape=(), dtype=float64)
tf.Tensor(31.42924901532083, shape=(), dtype=float64)
tf.Tensor(31.317642305835818, shape=(), dtype=float64)
tf.Tensor(31.193027421452875, shape=(), dtype=float64)
tf.Tensor(31.05511329577808, shape=(), dtype=float64)
tf.Tensor(30.903670947416003, shape=(), dtype=float64)
tf.Tensor(30.738518595844994, shape=(), dtype=float64)
tf.Tensor(30.559472268761304, shape=(), dtype=float64)
tf.Tensor(30.366241150896286, shape=(), dtype=float64)
tf.Tensor(30.158317435426234, shape=(), dtype=float64)
tf.Tensor(29.935018585753205, shape=(), dtype=float64)
tf.Tensor(29.695672832522273, shape=(), dtype=float64)
tf.Tensor(29.439699409189803, shape=(), dtype=float64)
tf.Tensor(29.166577690676892, shape=(), dtype=float64)
tf.Tensor(28.875852667036845, shape=(), dtype=float64)
tf.Tensor(28.56717138370702, shape=(), dtype=float64)
tf.Tensor(28.24

  0%|          | 0/200 [00:00<?, ?it/s]

tf.Tensor(31.26214720074752, shape=(), dtype=float64)
tf.Tensor(31.097847853719436, shape=(), dtype=float64)
tf.Tensor(30.91363091804328, shape=(), dtype=float64)
tf.Tensor(30.708199445376557, shape=(), dtype=float64)
tf.Tensor(30.479382687517003, shape=(), dtype=float64)
tf.Tensor(30.225409850239572, shape=(), dtype=float64)
tf.Tensor(29.945075852800507, shape=(), dtype=float64)
tf.Tensor(29.637351108253842, shape=(), dtype=float64)
tf.Tensor(29.30130456288041, shape=(), dtype=float64)
tf.Tensor(28.936110403114874, shape=(), dtype=float64)
tf.Tensor(28.54105962429776, shape=(), dtype=float64)
tf.Tensor(28.11557351990509, shape=(), dtype=float64)
tf.Tensor(27.6592135218838, shape=(), dtype=float64)
tf.Tensor(27.171708810328447, shape=(), dtype=float64)
tf.Tensor(26.653004579679717, shape=(), dtype=float64)
tf.Tensor(26.10335074861402, shape=(), dtype=float64)
tf.Tensor(25.52341161953133, shape=(), dtype=float64)
tf.Tensor(24.914365329141603, shape=(), dtype=float64)
tf.Tensor(24.277970

In [ ]:
T = 1
print(state_fidelity(channel_model1.choi(T), channel_model2.choi(T))/16)
print(state_fidelity(channel_model2.choi(T), channel_model3.choi(T))/16)
print(state_fidelity(channel_model3.choi(T), channel_model1.choi(T))/16)

tf.Tensor(0.9992355154891903, shape=(), dtype=float64)
tf.Tensor(0.9992438481267214, shape=(), dtype=float64)
tf.Tensor(0.999996579649297, shape=(), dtype=float64)


In [ ]:
T = 0.5
print(state_fidelity(channel_model1.choi(T), channel_model2.choi(T))/16)
print(state_fidelity(channel_model2.choi(T), channel_model3.choi(T))/16)
print(state_fidelity(channel_model3.choi(T), channel_model1.choi(T))/16)

tf.Tensor(0.06142214586551805, shape=(), dtype=float64)
tf.Tensor(0.00017776152267865716, shape=(), dtype=float64)
tf.Tensor(0.17332206665032093, shape=(), dtype=float64)


In [ ]:
jump_operator = JumpOperator(4, trainable=False)

channel_model1.liouvillian.gamma = 0.5
channel_model1.liouvillian.JumpOperator = jump_operator

channel_model2.liouvillian.gamma = 0.5
channel_model2.liouvillian.JumpOperator = jump_operator

channel_model3.liouvillian.gamma = 0.5
channel_model3.liouvillian.JumpOperator = jump_operator

T = 1
print(state_fidelity(choi_target, channel_model1.choi(T))/16)
print(state_fidelity(choi_target, channel_model2.choi(T))/16)
print(state_fidelity(choi_target, channel_model3.choi(T))/16)

tf.Tensor(0.8842877691278705, shape=(), dtype=float64)
tf.Tensor(0.8836034351836707, shape=(), dtype=float64)
tf.Tensor(0.884272549531078, shape=(), dtype=float64)


## Polynomial Hamiltonian

In [3]:
T = 1
U = tf.convert_to_tensor([[1, 0, 0, 0],
                            [0, 1, 0, 0],
                            [0, 0, 0, 1],
                            [0, 0, 1, 0]], dtype=tf.complex128)

channel_target = ChoiMapStatic(U, mode="unitary")
choi_target = channel_target.choi
n = 2
d = 2**n

tf.random.set_seed(42)
np.random.seed(42)
jump_operator = JumpOperator(d, trainable=False, trace_less=True)

H_model1 = PolynomialHamiltonian(d=d, degree=1)

lindblad_model1 = LindbladGenerator(hamiltonian = H_model1, 
                                   jump_operator = jump_operator,
                                   gamma = 0.0)

channel_model1 = MagnusPropagator(liouvillian=lindblad_model1, grid_size=200)
print(tf.math.reduce_sum(tf.abs(channel_model1.choi(T) - choi_target)**2))

tf.Tensor(30.682355812857665, shape=(), dtype=float64)


In [4]:
T = 1

optimizer = tf.optimizers.Adam(learning_rate=0.01)
for i in tqdm(range(200)):
    with tf.GradientTape() as tape:
        tape.watch(channel_model1.parameter_list)
        choi_model = channel_model1.choi(T)
        loss = tf.math.reduce_sum(tf.abs(channel_model1.choi(T) - choi_target) ** 2)

    print(loss)
    gradients = tape.gradient(loss, channel_model1.parameter_list)
    optimizer.apply_gradients(zip(gradients, channel_model1.parameter_list))

  0%|          | 0/200 [00:00<?, ?it/s]

tf.Tensor(30.682355812857665, shape=(), dtype=float64)
tf.Tensor(30.23675682932933, shape=(), dtype=float64)
tf.Tensor(29.73886004593071, shape=(), dtype=float64)
tf.Tensor(29.19011868776696, shape=(), dtype=float64)
tf.Tensor(28.590956367711392, shape=(), dtype=float64)
tf.Tensor(27.944166156664934, shape=(), dtype=float64)
tf.Tensor(27.25218902480441, shape=(), dtype=float64)
tf.Tensor(26.517088416194817, shape=(), dtype=float64)
tf.Tensor(25.741701363593954, shape=(), dtype=float64)
tf.Tensor(24.92939667852636, shape=(), dtype=float64)
tf.Tensor(24.083214003028054, shape=(), dtype=float64)
tf.Tensor(23.205778304932277, shape=(), dtype=float64)
tf.Tensor(22.300036397470194, shape=(), dtype=float64)
tf.Tensor(21.369658681217658, shape=(), dtype=float64)
tf.Tensor(20.41874449468426, shape=(), dtype=float64)
tf.Tensor(19.451388477721487, shape=(), dtype=float64)
tf.Tensor(18.471445136491834, shape=(), dtype=float64)
tf.Tensor(17.482534190822843, shape=(), dtype=float64)
tf.Tensor(16.488

In [5]:
print(state_fidelity(choi_target, channel_model1.choi(T))/16)
lindblad_model1.gamma = 1
print(state_fidelity(choi_target, channel_model1.choi(T))/16)

tf.Tensor(1.0000000416982169, shape=(), dtype=float64)
tf.Tensor(0.86482563705173, shape=(), dtype=float64)


In [7]:
T = 1
U = tf.convert_to_tensor([[1, 0, 0, 0],
                            [0, 1, 0, 0],
                            [0, 0, 0, 1],
                            [0, 0, 1, 0]], dtype=tf.complex128)

channel_target = ChoiMapStatic(U, mode="unitary")
choi_target = channel_target.choi
n = 2
d = 2**n

tf.random.set_seed(42)
np.random.seed(42)
jump_operator = JumpOperator(d, trainable=False, trace_less=True)

H_model2 = PolynomialHamiltonian(d=d, degree=1)

lindblad_model2 = LindbladGenerator(hamiltonian = H_model2, 
                                    jump_operator = jump_operator,
                                    gamma = 1)

channel_model2 = MagnusPropagator(liouvillian=lindblad_model2, grid_size=200)
print(tf.math.reduce_sum(tf.abs(channel_model2.choi(T) - choi_target)**2))

tf.Tensor(27.058096817906822, shape=(), dtype=float64)


In [8]:
T = 1

#optimizer = tf.optimizers.Adam(learning_rate=0.01)
for i in tqdm(range(200)):
    with tf.GradientTape() as tape:
        tape.watch(channel_model2.parameter_list)
        choi_model = channel_model2.choi(T)
        loss = tf.math.reduce_sum(tf.abs(channel_model2.choi(T) - choi_target) ** 2)

    print(loss)
    gradients = tape.gradient(loss, channel_model2.parameter_list)
    optimizer.apply_gradients(zip(gradients, channel_model2.parameter_list))

  0%|          | 0/200 [00:00<?, ?it/s]

tf.Tensor(27.058096817906822, shape=(), dtype=float64)
tf.Tensor(26.554640261432926, shape=(), dtype=float64)
tf.Tensor(25.73875468866531, shape=(), dtype=float64)
tf.Tensor(24.616038848891375, shape=(), dtype=float64)
tf.Tensor(23.212747700332155, shape=(), dtype=float64)
tf.Tensor(21.570554194954315, shape=(), dtype=float64)
tf.Tensor(19.74908259032167, shape=(), dtype=float64)
tf.Tensor(17.80836790188424, shape=(), dtype=float64)
tf.Tensor(15.789713007998712, shape=(), dtype=float64)
tf.Tensor(13.70668522925843, shape=(), dtype=float64)
tf.Tensor(11.568025918816392, shape=(), dtype=float64)
tf.Tensor(9.42650126542786, shape=(), dtype=float64)
tf.Tensor(7.377430922514176, shape=(), dtype=float64)
tf.Tensor(5.526377194224363, shape=(), dtype=float64)
tf.Tensor(3.9555116097307867, shape=(), dtype=float64)
tf.Tensor(2.708714992393487, shape=(), dtype=float64)
tf.Tensor(1.796846683911209, shape=(), dtype=float64)
tf.Tensor(1.2047741279013897, shape=(), dtype=float64)
tf.Tensor(0.89744215

In [9]:
print(state_fidelity(choi_target, channel_model2.choi(T))/16)

tf.Tensor(0.8691460155308343, shape=(), dtype=float64)


## Observations

Dynamical Hamiltonian can't mitigate noise from time-independent jump operators

Different realizations of dynamical Hamiltonians with same overall effect -> add noise, same degredation, that is, influence is path independent.

Remember, time is static. Length of path is very important for total noise, exact path is maybe not.

Here, noise is traceless jump operators -> no part can be absorbed into hamiltonian. Hamiltonian part of noise can be trivially accounted for, still valuable tho.
What about time dependent noise? Non-markovian noise?